IMPORT DEPENDENCIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

PUT DATA IN PANDAS DATA FRAME

In [2]:
stock_dat = pd.read_csv('/content/StockDataInt.csv')

VIEW DATA

In [3]:
stock_dat.head()


,Unnamed: 0,Entry_date,Entry_time,Type,Entry_price,Exit_price,Exit_time,Profit,Reason
0,0,2011-01-04,09:30:00,Short,11822.00,11611.200,15:20:00,206.0712,EOD
1,1,2011-01-05,09:40:00,Short,11474.00,11382.000,15:20:00,87.4104,EOD
2,2,2011-01-06,13:50:00,Short,11259.75,11304.789,14:15:00,-49.5429,SL
3,3,2011-01-06,14:50:00,Short,11253.00,11251.300,15:20:00,-2.8012,EOD
4,4,2011-01-07,09:50:00,Short,11150.00,11194.600,11:45:00,-49.0600,SL


FEATURE ENCODE

1.   IN 'Type' Short = 1 , Long = 0
2.   IN 'Reason' EOD = 1, SL = 0, Target = 2
3.   IN 'Profit' profit(+ve) = 1, profit(-ve) = 0 


In [4]:
stock_dat['Type'] = stock_dat['Type'].apply(lambda x : 1 if x == 'Short' else 0)
stock_dat['Reason'] = stock_dat['Reason'].apply(lambda x : 1 if x == 'EOD' else (0 if x == 'SL' else (2 if x == 'Target' else None)))
stock_dat['Profit'] = stock_dat['Profit'].apply(lambda x : 1 if x >= 0 else 0)

CHECK DATA FOR ENCODINGS

In [5]:
stock_dat.head()

,Unnamed: 0,Entry_date,Entry_time,Type,Entry_price,Exit_price,Exit_time,Profit,Reason
0,0,2011-01-04,09:30:00,1,11822.00,11611.200,15:20:00,1,1
1,1,2011-01-05,09:40:00,1,11474.00,11382.000,15:20:00,1,1
2,2,2011-01-06,13:50:00,1,11259.75,11304.789,14:15:00,0,0
3,3,2011-01-06,14:50:00,1,11253.00,11251.300,15:20:00,0,1
4,4,2011-01-07,09:50:00,1,11150.00,11194.600,11:45:00,0,0


CODE TO REPLACE THE ':' FROM THE 'Entry_time' and 'Exit_time'

WE CREATE A NEW FEATURE HERE :

1. 'Period' = 'Exit_time/100' - 'Entry_time/100'
2.  This is the total priod in minutes for which the stock was invested.




In [6]:
stock_dat['Entry_time'] = stock_dat['Entry_time'].astype(str)
stock_dat["Entry_time"] = stock_dat["Entry_time"].replace(":","", regex=True)
stock_dat["Exit_time"] = stock_dat["Exit_time"].replace(":","", regex=True)
stock_dat["Entry_time"] = stock_dat["Entry_time"].astype(int)
stock_dat["Exit_time"] = stock_dat["Exit_time"].astype(int)
stock_dat["Entry_time"] = stock_dat["Entry_time"]/100
stock_dat["Exit_time"] = stock_dat["Exit_time"]/100
stock_dat["Period"] = stock_dat["Exit_time"] - stock_dat["Entry_time"]

CHECK DATAFRAME FOR THE NEW FEATURES

In [7]:
stock_dat.head()

,Unnamed: 0,Entry_date,Entry_time,Type,Entry_price,Exit_price,Exit_time,Profit,Reason,Period
0,0,2011-01-04,930.0,1,11822.00,11611.200,1520.0,1,1,590.0
1,1,2011-01-05,940.0,1,11474.00,11382.000,1520.0,1,1,580.0
2,2,2011-01-06,1350.0,1,11259.75,11304.789,1415.0,0,0,65.0
3,3,2011-01-06,1450.0,1,11253.00,11251.300,1520.0,0,1,70.0
4,4,2011-01-07,950.0,1,11150.00,11194.600,1145.0,0,0,195.0


CODE TO REPLACE THE '-' FROM THE 'Entry_date'






In [8]:
stock_dat['Entry_date'] = stock_dat['Entry_date'].astype(str)
stock_dat["Entry_date"] = stock_dat["Entry_date"].replace("-","", regex=True)

In [9]:
stock_dat['Entry_date'] = stock_dat['Entry_date'].astype(int)

CHECK DATAFRAME FOR CHANGES

In [10]:
stock_dat.head()

,Unnamed: 0,Entry_date,Entry_time,Type,Entry_price,Exit_price,Exit_time,Profit,Reason,Period
0,0,20110104,930.0,1,11822.00,11611.200,1520.0,1,1,590.0
1,1,20110105,940.0,1,11474.00,11382.000,1520.0,1,1,580.0
2,2,20110106,1350.0,1,11259.75,11304.789,1415.0,0,0,65.0
3,3,20110106,1450.0,1,11253.00,11251.300,1520.0,0,1,70.0
4,4,20110107,950.0,1,11150.00,11194.600,1145.0,0,0,195.0


WE CREATE A NEW FEATURE HERE :

1. 'Interday' = 'Entry_date[sample]' - 'Entry_date[sample-1]'
2.  If the difference = 0 then 'Interday' = 0 else 'Interday' = 1
3.  The values for the first and last day is assumed to be 0
4.  This is to differentiate Interday trades from Intraday trades

In [11]:
stock_dat['Interday'] = stock_dat['Entry_date']
for i in range(0, 3349):
    if i == 0 or i == 3348:
      stock_dat['Interday'][i] = 0
    else:
      stock_dat['Interday'][i] = stock_dat['Entry_date'][i] - stock_dat['Entry_date'][i-1]
      if stock_dat['Interday'][i] > 0:
         stock_dat['Interday'][i] = 1
      
        

<ipython-input-11-0caa3150bf5e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_dat['Interday'][i] = 0
<ipython-input-11-0caa3150bf5e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_dat['Interday'][i] = stock_dat['Entry_date'][i] - stock_dat['Entry_date'][i-1]
<ipython-input-11-0caa3150bf5e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_dat['Interday'][i] = 1


CHECK FOR THE CHANGES AND NEW FEATURES

In [12]:
stock_dat.head()

,Unnamed: 0,Entry_date,Entry_time,Type,Entry_price,Exit_price,Exit_time,Profit,Reason,Period,Interday
0,0,20110104,930.0,1,11822.00,11611.200,1520.0,1,1,590.0,0
1,1,20110105,940.0,1,11474.00,11382.000,1520.0,1,1,580.0,1
2,2,20110106,1350.0,1,11259.75,11304.789,1415.0,0,0,65.0,1
3,3,20110106,1450.0,1,11253.00,11251.300,1520.0,0,1,70.0,0
4,4,20110107,950.0,1,11150.00,11194.600,1145.0,0,0,195.0,1


NOW WE DROP THE VARIOUS FEATURES THAT EITHER WE DON'T NEED LIKE THE FIRST INDEX COLUMN 'Unnamed: 0' AND THE COLUMNS THAT WE HAVE USED TO CREATE NEW FEATURES LIKE 'Period' and 'Interday'

In [13]:
stock_dat_final = pd.DataFrame()
stock_dat_final = stock_dat.drop(columns = ['Unnamed: 0', 'Entry_date', 'Entry_time', 'Exit_time'], axis = 1)

VIEW THE NEW DATA SET

In [14]:
stock_dat_final.describe()

,Type,Entry_price,Exit_price,Profit,Reason,Period,Interday
count,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000
mean,0.490594,21547.890251,21550.117552,0.375933,0.482234,236.270529,0.648850
std,0.499986,9686.451215,9688.114089,0.484435,0.510403,213.558362,0.477401
min,0.000000,7917.900000,7937.300000,0.000000,0.000000,5.000000,0.000000
25%,0.000000,11964.000000,11930.000000,0.000000,0.000000,65.000000,0.000000
50%,0.000000,19422.000000,19364.580600,0.000000,0.000000,165.000000,1.000000
75%,1.000000,29156.000000,29148.000000,1.000000,1.000000,415.000000,1.000000
max,1.000000,44166.450000,44172.400000,1.000000,2.000000,605.000000,1.000000


NOW WE NORMALISE THE DATA BY DIVIDNG WITH THE MAX DATA VALUE IN THE COLUMN THIS ALLOWS FOR US TO BRING ALL FEATURES IN THE INCLUSIVE RANGE OF [0,1]. THIS ALLOWS THE GRADIENT DESCENT IN THE REGRESSION MODEL TO WORK.

In [15]:
stock_dat_final['Entry_price'] = stock_dat_final['Entry_price']/44166
stock_dat_final['Period'] = stock_dat_final['Period']/605
stock_dat_final['Exit_price'] = stock_dat_final['Exit_price']/44172

VIEW THE DATA TO CHECK CHANGES

In [16]:
stock_dat_final.head()

,Type,Entry_price,Exit_price,Profit,Reason,Period,Interday
0,1,0.267672,0.262863,1,1,0.975207,0
1,1,0.259793,0.257675,1,1,0.958678,1
2,1,0.254942,0.255927,0,0,0.107438,1
3,1,0.254789,0.254716,0,1,0.115702,0
4,1,0.252457,0.253432,0,0,0.322314,1


NOW WE SEPARTE THE LABELS FORM THE FEATURES

In [17]:
X = stock_dat_final.drop('Profit', axis = 1)
Y = stock_dat_final['Profit']

FINAL CHECK FOR ANY NULL VALUES

In [18]:
print(stock_dat_final.isnull().sum())

Type           0
Entry_price    0
Exit_price     0
Profit         0
Reason         0
Period         0
Interday       0
dtype: int64


SPLIT THE TRAIN AND TEST DATA IN THE RATIO 90 : 10

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)


NOW WE SUMMON THE LOGISTIC REGRESSION ESTIMATOR CLASS INTO THE MODEL VARIABLE AND TRAIN IT ON THE TRAIN DATA

In [20]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

NOW WE USE THE TEST DATA TO MAKE PREDICTIONS OF THE PROFIT AND NON PROFIT TRADES

In [21]:
Y_predict = model.predict(X_test)
accuracy = accuracy_score(Y_predict, Y_test)
print(accuracy*100)

89.55223880597015


WE OBSERVE THAT THE MODEL WAS SUCCESFULLY ABLE TO PREDICT APPROXIMATELY 92% TIMES THE PROFITABLE TRADES FROM THE NON - PROFITABLE ONES

NOW WE USE CROSS VALIDATION FOR MORE ROBUST TESTING OF THE MODEL OVER DIFFERENT PERMUTATIONS AND COMBINATIONS OF DATA

In [22]:
scores = cross_val_score(model, X, Y, cv=5)
for i in range(0, 5):
  print('THE SCORE FOR ', i + 1, 'CROSS VALIDATION SET IS : ', "{:.2f}".format(scores[i] * 100),'% ACCURACY')

THE SCORE FOR  1 CROSS VALIDATION SET IS :  91.34 % ACCURACY
THE SCORE FOR  2 CROSS VALIDATION SET IS :  93.43 % ACCURACY
THE SCORE FOR  3 CROSS VALIDATION SET IS :  82.69 % ACCURACY
THE SCORE FOR  4 CROSS VALIDATION SET IS :  91.64 % ACCURACY
THE SCORE FOR  5 CROSS VALIDATION SET IS :  89.84 % ACCURACY


SCORES OF 5 DIFFERENT PERMUTATION AND COMBINATION THE THE DATA